# 14 分布式Tensorflow

## 14.2 分布式原理

TensorFlow的分布式集群由多个服务器进程和客户端进程组成。TensorFlow有几种部署方式，如单机多卡和分布式（多机多卡），一般我们把多机多卡的部署称为TensorFlow的分布式。

### 14.1.1 单机多卡和分布式

&emsp;&emsp;单机多卡是指单台服务器有多块GPU。假设一台机器上有4快GPU，单机多GPU的训练过程如下。  
（1）在单机单GPU的训练中，数据是一个批次（batch）一个批次的训练的。在单机多GPU中，一次处理4个批次的数据，每个GPU处理一个批次的数据计算。   
（2）变量，也就是参数，保存在CPU上，数据由CPU分发给4个GPU，在GPU上完成计算，得到每个批次要更新的梯度。   
（3）在CPU上收集完4个GPU上要更新的梯度，计算一下平均梯度，然后更新参数。  
（4）继续第2步和第3步，循环这个过程。  
&emsp;&emsp;这个过程的处理速度取决于最慢的那个GPU的速度。如果4个GPU的处理速度差不多，处理速度就相当于单GPU的速度的4倍减去数据在CPU和GPU之间传输的开销，但是，这样进行并行训练，运算能力还是限制在单机上。  
&emsp;&emsp;分布式是指训练在多个工作节点（worker)上。工作节点是指实现计算的一个单元，如果计算服务器是单卡，一般就是指这台服务器；如果计算服务器是多卡，还可以根据多个GPU划分多个工作节点。当数据量大到超过一台机器的处理能力时，必须使用分布式。  
&emsp;&emsp;分布式TensorFlow底层的通信是gRPC（google remote procedure call）。

### 14.1.2 分布式部署方式

在分布式运行的情况下，我们需要有多个计算单元（工作节点），后端的服务器可以部署为单工作节点和多工作节点。

**1.单工作节点部署**

&emsp;&emsp;单工作节点部署是在每台服务器上运行一个工作节点，假设服务器有4个GPU，一个工作节点可以访问4块GPU卡，这时需要在代码中使用tf.device()指定运行操作的设备。  
&emsp;&emsp;单工作节点部署的优势是在单机多个GPU间需要通信的情况下，效率更高。例如，可以实现RNN的模型并行。单工作节点部署的劣势是需要手动在代码中指定设备。

**2.多工作节点部署**

(1)设置CUDA_VISIBLE_DEVICES环境变量，限制各个工作节点只可见一个GPU，启动进程时添加环境变量即可。例如，每个工作节点只能访问一个GPU，在代码中不需要额外指定。https://github.com/tobegit3hub/tensorflow_examples/tree/master/distributed_tensorflow

In [1]:
CUDA_VISIBLE_DEVICES='' python ./distributed_supervisor.py --ps_hosts=127.0.0.1:2222,127.0.0.1:2223 --worker_hosts=127.0.0.1:2224,127.0.0.1:2225 --job_name=ps --task_index=0
CUDA_VISIBLE_DEVICES='' python ./distributed_supervisor.py --ps_hosts=127.0.0.1:2222,127.0.0.1:2223 --worker_hosts=127.0.0.1:2224,127.0.0.1:2225 --job_name=ps --task_index=1
CUDA_VISIBLE_DEVICES='0' python ./distributed_supervisor.py --ps_hosts=127.0.0.1:2222,127.0.0.1:2223 --worker_hosts=127.0.0.1:2224,127.0.0.1:2225 --job_name=worker --task_index=0
CUDA_VISIBLE_DEVICES='1' python ./distributed_supervisor.py --ps_hosts=127.0.0.1:2222,127.0.0.1:2223 --worker_hosts=127.0.0.1:2224,127.0.0.1:2225 --job_name=worker --task_index=1

SyntaxError: invalid syntax (<ipython-input-1-43c926469507>, line 1)

(2)使用tf.device()指定使用特定的GPU。

多工作节点部署的优势是代码简单，提高GPU使用率。多工作节点部署的劣势是工作节点间如果需要通信就不能利用本地GPU通信的优势，而且部署时需要部署多个工作节点。

## 14.2 分布式架构

分布式架构主要由客户端和服务端组成，服务端又包括主节点(master)和工作节点(worker)两者组成。

### 14.2.1 客户端、主节点和工作节点的关系

简单来说，客户端通过会话来联系主节点，实际的工作交由工作节点实现。每个工作节点占据一台设别（CPU,GPU）。在单机模式下，客户端、主节点和工作节点都在同一台服务器上；在分布式模式下，它们可以位于不同的服务器上。

**1.客户端**

客户端用于建立Tensorflow计算图，并建立与集群进行交互的会话层。因此，代码中只要包含Session()就是客户端。

**2.服务端**

服务端是一个运行了tf.train.Server实例的进程，是TensorFlow执行任务的集群(cluster)的一部分，并有主节点服务（Master Service)和工作节点服务之分。运行中由一个主节点进程组成，主节点进程和工作节点进程之间通过接口通信。单机多卡和分布式都是这种结构，因此只需要更改它们之间通信的接口就可以实现单机多卡和分布式的切换。

**3.主节点服务**

主节点服务实现了tensorflow:Session接口，通过RPC服务程序来远程连接工作节点，与工作节点的服务进程中的工作任务进行通信。在TensorFlow服务端中，一般是task_index为0的作业（job）

**4.工作节点服务**

工作节点服务实现了worker_service.proto接口，使用本地设备对部分图进行计算。在TensorFlow服务端中，所有工作节点都包含工作节点的服务逻辑。工作节点也可以是本地不同端口的不通进程，或者多台服务器上的多个进程。